In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization, Rescaling
from tensorflow.keras.layers import RandomContrast, RandomZoom, RandomFlip, RandomRotation, RandomTranslation, RandomCrop, RandomBrightness, GaussianNoise
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, EarlyStopping
from sklearn.metrics import confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay
from absl import logging

logging.set_verbosity(logging.ERROR)
np.set_printoptions(suppress=True)
AUTOTUNE = tf.data.AUTOTUNE


# Load the Dataset

In [ ]:
# Menggunakan tanda seru (!) untuk menjalankan perintah shell
!git clone https://github.com/GreenAvo-Capstone/Machine-Learning.git

Cloning into 'Machine-Learning'...
remote: Enumerating objects: 15048, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 15048 (delta 14), reused 307 (delta 13), pack-reused 14739
Receiving objects: 100% (15048/15048), 412.13 MiB | 18.07 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Updating files: 100% (29424/29424), done.


In [ ]:
import os

# Cek apakah direktori dataset sudah ada
dataset_dir = './Machine-Learning/dataset'
if os.path.exists(dataset_dir):
    print("Dataset berhasil diunduh.")
else:
    print("Dataset tidak ditemukan.")


Dataset berhasil diunduh.


In [ ]:
classes_name = os.listdir(dataset_dir)
dict_disease = {}

for number, name in enumerate(classes_name):
    if name == 'filename': continue
    dict_disease[number] = name.strip()
dict_disease = {key: dict_disease[key] for key in dict_disease if key != 'Unlabeled'}
img_size = 224

path_train = os.path.join(dataset_dir, 'train')
path_valid = os.path.join(dataset_dir, 'val')
path_test = os.path.join(dataset_dir, 'test')

# Memeriksa apakah direktori valid dan test ada
if not os.path.exists(path_train):
    raise FileNotFoundError(f"Training directory not found: {path_train}")
if not os.path.exists(path_valid):
    raise FileNotFoundError(f"Validation directory not found: {path_valid}")
if not os.path.exists(path_test):
    raise FileNotFoundError(f"Test directory not found: {path_test}")

# Mengatur AUTOTUNE
AUTOTUNE = tf.data.AUTOTUNE

# **Dynamically determine classes:**
all_classes = set()
for dataset_path in [path_train, path_valid, path_test]:
    for class_name in os.listdir(dataset_path):
        if class_name != 'filename':
            all_classes.add(class_name)

dict_disease = {i: class_name for i, class_name in enumerate(sorted(all_classes))}

with tf.device('/cpu:0'):
    train_dataset = tf.keras.utils.image_dataset_from_directory(path_train, shuffle=False, batch_size=None, image_size=(img_size, img_size))
    valid_dataset = tf.keras.utils.image_dataset_from_directory(path_valid, shuffle=False, batch_size=None, image_size=(img_size, img_size))
    test_dataset = tf.keras.utils.image_dataset_from_directory(path_test, shuffle=False, batch_size=None, image_size=(img_size, img_size))

    test_labels = []
    for _, label in test_dataset:
        test_labels.append(label.numpy())
    test_labels = np.array(test_labels).reshape(-1)

Found 11032 files belonging to 5 classes.
Found 2205 files belonging to 5 classes.
Found 1473 files belonging to 5 classes.


In [ ]:
with tf.device('/cpu:0'):
    train = train_dataset.cache('train_dataset').repeat(1).shuffle(256, reshuffle_each_iteration=True).batch(32).prefetch(AUTOTUNE)
    valid = valid_dataset.cache('valid-dataset').batch(32).prefetch(AUTOTUNE)
    test = test_dataset.cache('test-dataset').batch(32).prefetch(AUTOTUNE)

In [ ]:
def confusionMatrix(epoch, logs):
    test_loss, test_acc, topK_acc = model.evaluate(test, verbose=0)
    print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc*100:.2f}%\nTopK Accuracy: {topK_acc*100:.2f}%")

    yhat = model.predict(test, verbose=0)
    yhat = np.argmax(yhat, axis=1)

    # Check the number of unique labels in the test dataset
    unique_labels = np.unique(test_labels)
    cm_labels = [dict_disease[i] for i in unique_labels]

    # Plotting confusion matrix
    disp = ConfusionMatrixDisplay.from_predictions(
        test_labels,
        yhat,
        display_labels=cm_labels,
        xticks_rotation=45
    )

    # Adjust the number of tick locations
    disp.ax_.set_xticks(np.arange(len(cm_labels)))
    disp.ax_.set_xticklabels(cm_labels, rotation=45)
    disp.ax_.set_yticks(np.arange(len(cm_labels)))
    disp.ax_.set_yticklabels(cm_labels)

    plt.title(f"Confusion Matrix on epoch {epoch+1}\nVal accuracy: {logs.get('val_accuracy'):.3f}\nTest accuracy: {test_acc:.3f}")

    try:
        os.makedirs("./CM/")
    except FileExistsError:
        pass

    plt.savefig(f"./CM/Epoch {epoch+1}.png", dpi=300, bbox_inches='tight')
    plt.close('all')


# Making the Machine Learning Model

In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(input_shape=(img_size,img_size,3), include_top=False, include_preprocessing=False, pooling='max')
base_model.trainable=False

model = Sequential([
    Input((img_size, img_size, 3), dtype=tf.float32),
    GaussianNoise(stddev=0.5),
    Rescaling(1/255),
    RandomFlip(),
    RandomRotation(factor=0.3),
    RandomZoom(height_factor=(-0.2, 0.2)),
    RandomBrightness(factor=(-0.2,0.2), value_range=[0,1]),
    base_model,
    Dense(128, activation='selu'),
    # Make sure the final Dense layer matches the number of classes:
    Dense(len(dict_disease))
])

model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, amsgrad=True),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)]
)

bestCB = ModelCheckpoint(filepath='./checkpoint/best/', monitor='val_accuracy',
                         mode='max', verbose=1, save_best_only=True)

cmCB = LambdaCallback(on_epoch_end=confusionMatrix)

esCB = EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)

model.summary()


# Train the Model

In [ ]:
with tf.device('/gpu:0'):
    history = model.fit(
        train,
        validation_data=valid,
        epochs=1000,
        callbacks=[bestCB, cmCB, esCB]
    )

In [ ]:
model.save("./checkpoint/latest/")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
topK_acc = history.history['sparse_top_k_categorical_accuracy']
val_topK_acc = history.history['val_sparse_top_k_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = history.epoch

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Model's Accuracy")
plt.legend()
plt.show()

plt.plot(epochs, topK_acc, label='Train TopK Accuracy')
plt.plot(epochs, val_topK_acc, label='Validation TopK Accuracy')
plt.xlabel("Epochs")
plt.ylabel("TopK Accuracy")
plt.title("Model's TopK Accuracy")
plt.legend()
plt.show()

plt.plot(epochs[10:], loss[10:], label='Train Loss')
plt.plot(epochs[10:], val_loss[10:], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model's Loss")
plt.legend()
plt.show()

# Evaluate the Model Created

In [ ]:
# Load iterasi terakhir
# model = tf.keras.models.load_model("./checkpoint/latest/")

# Load terbaik
model = tf.keras.models.load_model("./checkpoint/best/")

In [ ]:
test_loss, test_acc, topK_acc = model.evaluate(test, verbose=0)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc*100:.2f}%\nTopK Accuracy: {topK_acc*100:.2f}%")

In [ ]:
topK = 3 # Ambil 3 kategori tertinggi untuk display ke user

for number, (image, label) in enumerate(test_dataset.shuffle(512).take(20)):
    image = tf.cast(image, tf.uint8)

    plt.imshow(image, interpolation='nearest')
    plt.title(dict_disease[label.numpy()])
    plt.axis('off')

    pred_image      = tf.cast(image, tf.float32)
    pred_image      = tf.expand_dims(image, axis=0)
    yhat            = tf.nn.softmax(model.predict(pred_image, verbose=0))
    yhat_topk       = np.argpartition(-yhat, topK-1)[0][:3]
    true_label      = dict_disease[label.numpy()]
    prob_res        = []
    disease_types   = []
    text_result     = []

    for disease in yhat_topk:
        disease_types.append(dict_disease[disease])
        prob_res.append(yhat[0][disease])

    for index_disease, disease in enumerate(disease_types):
        res = "benar" if true_label == disease else "salah"
        res = f"{index_disease+1}: {disease} ({res}), {prob_res[index_disease] * 100:.2f}%."
        text_result.append(res)

    plt.figtext(0.5, 0.01, "\n".join(text_result), ha='center', wrap=True)
    plt.savefig(f"Prediction {number+1}.png", transparent=True)
    plt.show()

# Convert the Model to TensorFlow Lite

In [ ]:
# Optimize size
converter = tf.lite.TFLiteConverter.from_saved_model("./checkpoint/best/")
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

tflite_file = pathlib.Path('model size.tflite')
tflite_file.write_bytes(tflite_model)

In [ ]:
# Optimize latency
converter = tf.lite.TFLiteConverter.from_saved_model("./checkpoint/best/")
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
tflite_model = converter.convert()

tflite_file = pathlib.Path('model latency.tflite')
tflite_file.write_bytes(tflite_model)